In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import output

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
random.seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)
import time
start_time = time.time()

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')


%cd /content/gdrive/MyDrive/LonelinessR21
%ls

Mounted at /content/gdrive/
/content/gdrive/MyDrive/LonelinessR21
 annotation_Drew_sample_social_disconnection.xlsx
 annotation_Drew_sample_stigmatizing_labels.xlsx
 annotation_Selen_sample_social_disconnection.xlsx
 annotation_Selen_sample_stigmatizing_labels.xlsx
'Copy of loneliness_social_isolation_lexicon_evaluation.csv'
 gold_standard_loneliness_lexicon_df.csv
 gold_standard_social_isolation_1000.csv
 loneliness_ehr_roberta.ipynb
 loneliness_gold_standard_expanded_lexicon_matches.xlsx
 loneliness_lexicon_dev.ipynb
 loneliness_lexicon.Rmd
 loneliness_lexicon_stem_and_similar_round1.csv
 loneliness_lexicon_stem_and_similar_round2.csv
 loneliness_matches_expanded.xlsx
 loneliness_matches.xlsx
 loneliness_regex_matching_and_sample.ipynb
 loneliness_roberta_ehr_performance.gsheet
 loneliness_social_isolation_lexicon_evaluation.csv
'loneliness_social_isolation_lexicon_evaluation KL.csv'
'Matched Terms from Social Disconnection Lexicon.png'
 new_loneliness_phrases.csv
 predictions.csv
 r

In [ ]:
df_concat_filtered  = pd.read_csv("gold_standard_social_isolation_1000.csv")
df_concat_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          1000 non-null   int64  
 1   Sentence ID                         1000 non-null   int64  
 2   TEXT                                1000 non-null   object 
 3   Sentence                            1000 non-null   object 
 4   matched_term                        1000 non-null   object 
 5   chronic_social_disconnection_label  1000 non-null   float64
 6   lives_alone_label                   1000 non-null   float64
 7   acute_social_disconnection_label    1000 non-null   float64
 8   full_text                           1000 non-null   object 
 9   full_text_truncated                 1000 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 78.2+ KB


In [ ]:

# Rename 'full_text_truncated' to 'text'
df_concat_filtered = df_concat_filtered.rename(columns={'full_text_truncated': 'text', 'lives_alone_label':'label'})
df_concat_filtered.reset_index(drop=True, inplace=True)

# Create a new dataframe with the renamed 'text' column and 'chronic_social_disconnection_label'
chronic_df = df_concat_filtered[['label', 'text']]

# Display the first few rows of chronic_df
chronic_df.info()
chronic_df.head()
chronic_df.reset_index(drop=True, inplace=True)
unique_chronic_values = chronic_df['label'].value_counts().reset_index()

# Renaming columns for clarity
unique_chronic_values.columns = ['label', 'count']
unique_chronic_values




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1000 non-null   float64
 1   text    1000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


,label,count
0,0.0,812
1,1.0,188


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 5e-06
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        if index >= len(self.text):
            raise IndexError(f"Index {index} out of bounds for dataset of length {len(self.text)}")

        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


In [ ]:
train_size = 0.8
train_data=chronic_df.sample(frac=train_size,random_state=0)
test_data=chronic_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(chronic_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (1000, 2)
TRAIN Dataset: (800, 2)
TEST Dataset: (200, 2)


In [ ]:
train_data['label'].unique()

array([0., 1.])

In [ ]:
test_data['label'].unique()
test_data['text'].head()

,text
0,"Ischemic disease was excluded (no EKG changes,..."
1,As other etiologies for bladder wall thickenin...
2,"MAE, patient is cooparative with but seems to ..."
3,Social: Found down in apartment alone.</s>T/SI...
4,It may represent extension / recurrence of a t...


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig.from_pretrained("roberta-base")
config.output_attentions = True

class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base", config=config)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # Get the hidden states, pooler output, and attention weights
        last_hidden_state, pooler_output, all_attentions = self.l1(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        pooler = last_hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output, all_attentions



In [ ]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        # Extract the logits (output) from the returned tuple
        logits, attention_weights = model(ids, mask)

        # Use the logits when computing the loss
        loss = loss_function(logits, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(logits.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if _ % 5000 == 0:
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return


print(len(training_loader.dataset))



800


In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.7009543776512146
Training Accuracy: 31.25


50it [23:34, 28.29s/it]


The Total Accuracy for Epoch 0: 79.5
Training Loss Epoch: 0.5468260771036149
Training Accuracy Epoch: 79.5


0it [00:00, ?it/s]

...

The Total Accuracy for Epoch 8: 98.375
Training Loss Epoch: 0.0656135606765747
Training Accuracy Epoch: 98.375


0it [00:00, ?it/s]

Training Loss: 0.10602328181266785
Training Accuracy: 93.75


50it [22:59, 27.58s/it]

The Total Accuracy for Epoch 9: 98.0
Training Loss Epoch: 0.07357963291928173
Training Accuracy Epoch: 98.0


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            # Extract the logits (output) from the returned tuple
            logits, attention_weights = model(ids, mask)
            logits = logits.squeeze()

            # Use the logits when computing the loss and other operations
            loss = loss_function(logits, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(logits.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu



In [ ]:
## VALIDATION

In [ ]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")



1it [00:08,  8.06s/it]

Validation Loss: 0.015424137935042381
Validation Accuracy: 100.0


13it [01:40,  7.75s/it]


Validation Loss Epoch: 0.07134791528089689
Validation Accuracy Epoch: 97.5
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       164
           1       0.88      1.00      0.94        36

    accuracy                           0.97       200
   macro avg       0.94      0.98      0.96       200
weighted avg       0.98      0.97      0.98       200

Confusion Matrix:
[[159   5]
 [  0  36]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(1620), tensor(97), tensor(4...         0          0
1  [tensor(0), tensor(47874), tensor(35), tensor(...         0          0
2  [tensor(0), tensor(26369), tensor(3935), tenso...         0          0
3  [tensor(0), tensor(4688), tensor(6256), tensor...         0          0
4  [tensor(0), tensor(34440), tensor(31995), tens...         0          0


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:07,  7.48s/it]

Validation Loss: 0.04283013567328453
Validation Accuracy: 100.0


13it [01:40,  7.76s/it]


Validation Loss Epoch: 0.07175443615191258
Validation Accuracy Epoch: 97.5
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       164
           1       0.88      1.00      0.94        36

    accuracy                           0.97       200
   macro avg       0.94      0.98      0.96       200
weighted avg       0.98      0.97      0.98       200

Confusion Matrix:
[[159   5]
 [  0  36]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(5632), tensor(31798), tenso...         0          0
1  [tensor(0), tensor(574), tensor(3699), tensor(...         1          1
2  [tensor(0), tensor(713), tensor(16), tensor(14...         0          0
3  [tensor(0), tensor(18547), tensor(139), tensor...         0          0
4  [tensor(0), tensor(387), tensor(16908), tensor...         0          0
test accuracy = 97.50%
Elapsed Time: 14233.38 seconds


In [ ]:
## PREDICTIONS

In [ ]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)

        with torch.no_grad():
            logits, _ = model(ids, mask)  # Here we unpack the tuple

        big_val, big_idx = torch.max(logits.data, dim=1)  # Use logits instead of outputs
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return pd.DataFrame(df)



In [ ]:
predicted_classes = predict_on_dataframe(chronic_df, model, tokenizer, max_len=512)
print(predicted_classes)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


     label                                               text  predictions
0      0.0  The electronic pacer device overlying the left...            0
1      0.0  No attempts to get OOB alone.</s>87 year old f...            0
2      0.0  Neuro: Pt remains withdrawn, opens eyes sponta...            0
3      1.0  FWB'ing B LE Social / Occupational History: Pt...            1
4      0.0  Response: Plan: .H/O anxiety Assessment: Letha...            0
..     ...                                                ...          ...
995    1.0  Lives at home alone, has home aide come in eve...            1
996    0.0  POST EXTUBATION PT CALM AND WITHDRAWN.</s>MICU...            0
997    0.0  I spoke with [**First Name8 (NamePattern2) 862...            0
998    0.0  #5: Mom in for few mins this am alone.</s>NPN ...            0
999    0.0  When family here they think she is [** 467**] ...            0

[1000 rows x 3 columns]


<ipython-input-19-f46c646d3b1d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predictions'] = predictions


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Assuming predicted_classes is a DataFrame with 'label' and 'predictions' columns
true_labels = predicted_classes['label'].tolist()  # Extract 'label' column as list
predicted_labels = predicted_classes['predictions'].tolist()  # Extract 'predictions' column as list

# Function for bootstrapping a metric
def bootstrap_metric(y_true, y_pred, metric_func, n_iterations=1000):
    bootstrap_values = []
    n_samples = len(y_true)
    for _ in range(n_iterations):
        indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        bootstrap_true = np.array(y_true)[indices]
        bootstrap_pred = np.array(y_pred)[indices]
        value = metric_func(bootstrap_true, bootstrap_pred)
        bootstrap_values.append(value)
    return bootstrap_values

# Bootstrapping each metric for the positive class (label = 1)
bootstrap_f1_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: f1_score(y_true, y_pred, pos_label=1))
bootstrap_precision_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: precision_score(y_true, y_pred, pos_label=1))
bootstrap_recall_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: recall_score(y_true, y_pred, pos_label=1))
bootstrap_accuracy = bootstrap_metric(true_labels, predicted_labels, accuracy_score)

# Function to compute mean and confidence intervals
def mean_and_confidence_interval(data, alpha=0.05):
    mean_value = np.mean(data)
    lower_percentile = 100 * alpha / 2.
    upper_percentile = 100 * (1 - alpha / 2.)
    lower = np.percentile(data, lower_percentile)
    upper = np.percentile(data, upper_percentile)
    return mean_value, lower, upper

# Compute mean and confidence intervals for each metric
f1_mean, f1_lower, f1_upper = mean_and_confidence_interval(bootstrap_f1_pos)
precision_mean, precision_lower, precision_upper = mean_and_confidence_interval(bootstrap_precision_pos)
recall_mean, recall_lower, recall_upper = mean_and_confidence_interval(bootstrap_recall_pos)
accuracy_mean, accuracy_lower, accuracy_upper = mean_and_confidence_interval(bootstrap_accuracy)

# Print results
print(f"F1 Score: Mean={f1_mean:.3f}, 95% CI=({f1_lower:.3f}, {f1_upper:.3f})")
print(f"Precision: Mean={precision_mean:.3f}, 95% CI=({precision_lower:.3f}, {precision_upper:.3f})")
print(f"Recall: Mean={recall_mean:.3f}, 95% CI=({recall_lower:.3f}, {recall_upper:.3f})")
print(f"Accuracy: Mean={accuracy_mean:.3f}, 95% CI=({accuracy_lower:.3f}, {accuracy_upper:.3f})")




F1 Score: Mean=0.942, 95% CI=(0.918, 0.963)
Precision: Mean=0.894, 95% CI=(0.850, 0.934)
Recall: Mean=0.994, 95% CI=(0.981, 1.000)
Accuracy: Mean=0.977, 95% CI=(0.967, 0.985)


In [23]:
!pip install shap
#visualization shapley

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 24.8 MB/s eta 0:00:00


In [34]:
import shap
import torch
from transformers import RobertaTokenizer
import numpy as np

# Load the tokenizer and the pre-trained RoBERTa model (assumed already trained on your task)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Example sentences you want to explain
sentences = [
    "Past Medical History: DMII with retinopathy fatty liver obesity constipation COPD HL DJD atypical chest pain schizoaffective disorder substance abuse Social History: Lives alone in [Location Name].",
    "Pt was living at home alone prior to hospitalization."
]

# Tokenize the sentences
tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Extract input_ids and attention_mask for SHAP
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

# Move input tensors to the correct device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)  # Ensure the model is on the same device

# Create SHAP explainer with DeepExplainer for transformers
explainer = shap.DeepExplainer(model, input_ids)

# Explain the model's predictions
shap_values = explainer.shap_values(input_ids)

# Convert input IDs to tokens for visualization
feature_names = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

# Visualize the SHAP values for the first sentence
shap.summary_plot(shap_values[0], feature_names=feature_names)


TypeError: RobertaClass.forward() missing 1 required positional argument: 'attention_mask'

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Define a custom Dataset class
class CustomTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, return_tensors="pt")

# Placeholder for chronic_df (ensure it's a pandas DataFrame)
X = chronic_df['text'].values
y = chronic_df['label'].values

skf = StratifiedKFold(n_splits=5)

# Empty lists to store sample sizes and F1 scores
sample_sizes = []
f1_scores = []

# Main training loop (fixed to handle class indices correctly)
for sample_size in range(100, 1001, 100):
    f1_scores_sample = []

    for train_index, test_index in skf.split(X, y):
        # Select training and test data and ensure labels are integers
        train_texts = X[train_index[:sample_size]].tolist()
        test_texts = X[test_index].tolist()
        train_labels = y[train_index[:sample_size]].astype(int)  # Ensure labels are integers
        test_labels = y[test_index].astype(int)  # Ensure labels are integers

        # Tokenize the datasets
        train_encodings = tokenize_function(train_texts)
        test_encodings = tokenize_function(test_texts)

        # Create custom PyTorch datasets
        train_dataset = CustomTextDataset(train_encodings, train_labels)
        test_dataset = CustomTextDataset(test_encodings, test_labels)

        # Load the RoBERTa model for classification (2 classes)
        model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

        # Set up training arguments
        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
        )

        # Define Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset
        )

        # Train the model
        trainer.train()

        # Get predictions
        predictions = trainer.predict(test_dataset).predictions.argmax(-1)  # Use argmax to get class predictions

        # Calculate F1 score
        f1 = f1_score(test_labels, predictions, average='binary')
        f1_scores_sample.append(f1)

    # Store the mean F1 score for this sample size
    sample_sizes.append(sample_size)
    f1_scores.append(np.mean(f1_scores_sample))

# Create a DataFrame for the results
results_df = pd.DataFrame({'Sample Size': sample_sizes, 'F1 Score': f1_scores})

# Plot the results (as before)
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(results_df['Sample Size'], results_df['F1 Score'], marker='o', linestyle='-', color='b')
plt.title('Live Alone Sample Size vs F1 Score for RoBERTa Model')
plt.xlabel('Sample Size')
plt.ylabel('F1 Score')
plt.grid(True)
plt.savefig("live_alone_loneliness_sample_size_f1_roberta.png")

# Save results to CSV
results_df.to_csv("live_alone_loneliness_sample_size_vs_f1_score_roberta.csv", index=False)

